<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Market_Making_for_currencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
# Import the necessary packages
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [112]:
import warnings
warnings.filterwarnings('ignore')


In [113]:
# Create a list of tickers for currencies
currencies = ['EURUSD=X', 'EURUSD=X', 'GBPJPY=X', 'JPY=X', 'GBP=X', 'GBPAUD=X', 'GBPCAD=X', 'GBPCAD=X','GBPCHF=X','GBPCNY=X','GBPINR=X','GBPNOK=X','GBPQAR=X','GBPZAR=X','EURCHF=X','EURJPY=X','EURSEK=X','EURCHF=X','EURHUF=X','CNY=X','HKD=X','SGD=X','INR=X','MXN=X','PHP=X','IDR=X','THB=X','MYR=X','ZAR=X','RUB=X' ]

In [114]:
# Create a function that will full the adj close prices for the currencies
def download_currency_data(currency, start_date, end_date):
    data = yf.download(currency, start=start_date, end=end_date)
    return data['Adj Close']

In [154]:
# Simulated market environment:
class MarketMaker:
  def __init__(self, initial_investment, max_inventory, trading_amount, trading_cost=0):
    self.inventory = 0  # Starting inventory is zero
    self.max_inventory = max_inventory
    self.trading_amount = trading_amount
    self.trading_cost = trading_cost  # Trading cost as a fraction
    self.cash = initial_investment
    self.total_trades = 0
    self.profit = 0
    self.initial_investment = initial_investment
    self.inventory_history = []
    self.cash_history = []
    self.profit_history = []

  def simulate_market(self, price_data, liquidity_shocks):
    # Iterate through price data, simulating trades with dynamic spread
    for i, price in enumerate(price_data):
      # Determine if we're in a liquidity shock period
      if liquidity_shocks[i]:
        spread_percentage = random.uniform(0.0005, 0.001)  # Larger spreads during liquidity shocks (5 pip to 10 pip)
        execution_probability = 0.5  # Reduced execution probability during shocks (50% chance)
      else:
        spread_percentage = random.uniform(0.0001, 0.0005)  # Normal spreads (1 pip to 5 pip)
        execution_probability = 1.0  # Full execution probability in normal times

      # Calculate the spread dynamically as a percentage of the transaction value
      spread = price * spread_percentage

      # Bid and Ask Prices
      bid_price = price - spread / 2
      ask_price = price + spread / 2

      # Simulate a trade based on inventory levels and execution probability
      if random.random() <= execution_probability:
        self.execute_order(bid_price, ask_price)

      # Record inventory, cash, and profit levels
      self.inventory_history.append(self.inventory)
      self.cash_history.append(self.cash)
      self.profit_history.append(self.profit)

  def execute_order(self, bid_price, ask_price):
    # Simulate order execution with trading costs
    # Buying at the bid price (if inventory allows)
    if self.inventory < self.max_inventory:
      transaction_value = bid_price * self.trading_amount
      trading_fee = transaction_value * self.trading_cost  # Apply trading cost on the transaction value
      self.inventory += self.trading_amount
      self.cash -= (transaction_value + trading_fee)  # Deduct the cost plus the fee
      self.total_trades += 1

    # Selling at the ask price (if inventory is above minimum)
    if self.inventory > -self.max_inventory:
      transaction_value = ask_price * self.trading_amount
      trading_fee = transaction_value * self.trading_cost  # Apply trading cost on the transaction value
      self.inventory -= self.trading_amount
      self.cash += (transaction_value - trading_fee)  # Add the revenue minus the fee
      self.total_trades += 1
      profit_per_trade = (ask_price - bid_price) * self.trading_amount - 2 * trading_fee  # Net profit after fees
      self.profit += profit_per_trade  # Calculate profit accounting for fees

  def get_performance_metrics(self):
    # Calculate profit as a percentage of the initial investment
    profit_percentage = (self.profit / self.initial_investment) * 100
    return {'Final Inventory': self.inventory,'Final Cash': self.cash,'Total Trades': self.total_trades,'Profit': self.profit, 'Profit Percentage': profit_percentage}

  def plot_inventory_cash(self):
    # Plot inventory, cash, and profit levels over time
    plt.figure(figsize=(12, 8))

    plt.subplot(3, 1, 1)
    plt.plot(self.inventory_history, label='Inventory')
    plt.title('Inventory over Time')
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(self.cash_history, label='Cash', color='orange')
    plt.title('Cash over Time')
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(self.profit_history, label='Profit', color='green')
    plt.title('Profit over Time')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [163]:
# Parameters for Market Making
currency_tickers = currencies
start_date = '2022-09-25'
end_date = '2024-09-25'
initial_investment = 1000000000  # Initial investment of $1 billion per currency
max_inventory = 1000000  # Maximum units to hold in inventory
trading_amount = 100000  # Amount units to trade per transaction
trading_cost = 0  # 0% trading cost per transaction
num_simulations = 20  # Number of simulations for each currenc

In [164]:
# Generate random liquidity shocks for each currency
def generate_liquidity_shocks(price_data, shock_probability=0.01):
  # Randomly assign liquidity shocks over the time period (e.g., 1% chance for a shock on any given day)
  return np.random.choice([True, False], size=len(price_data), p=[shock_probability, 1 - shock_probability])


In [165]:
# Download data and simulate for multiple currencies
performance_results = []  # Initialize a list to store performance metrics for all currencies
for currency in currency_tickers:
  # Download price data for each currency
  price_data = download_currency_data(currency, start_date, end_date)
  price_data = price_data.values  # Convert to array for easier processing

  # Generate liquidity shocks for this currency
  liquidity_shocks = generate_liquidity_shocks(price_data)

  # Initialize variables to accumulate performance metrics
  cumulative_performance = {'Final Inventory': 0, 'Final Cash': 0,'Total Trades': 0,'Profit': 0,'Profit Percentage': 0,}

  for simulation in range(num_simulations):
    print(f"\nSimulating for {currency} - Run {simulation + 1}")
    market_maker = MarketMaker(initial_investment, max_inventory, trading_amount, trading_cost)
    market_maker.simulate_market(price_data, liquidity_shocks)

    # Collect performance metrics for this simulation
    performance_metrics = market_maker.get_performance_metrics()

    # Accumulate performance metrics
    cumulative_performance['Final Inventory'] += performance_metrics['Final Inventory']
    cumulative_performance['Final Cash'] += performance_metrics['Final Cash']
    cumulative_performance['Total Trades'] += performance_metrics['Total Trades']
    cumulative_performance['Profit'] += performance_metrics['Profit']
    cumulative_performance['Profit Percentage'] += performance_metrics['Profit Percentage']

    # Calculate average performance metrics over all simulations
    average_performance = {key: value / num_simulations for key, value in cumulative_performance.items()}
    average_performance['Currency'] = currency

    # Append average performance metrics for this currency
    performance_results.append(average_performance)

# Convert performance results to DataFrame
performance_df = pd.DataFrame(performance_results)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Simulating for EURUSD=X - Run 1

Simulating for EURUSD=X - Run 2

Simulating for EURUSD=X - Run 3

Simulating for EURUSD=X - Run 4

Simulating for EURUSD=X - Run 5

Simulating for EURUSD=X - Run 6

Simulating for EURUSD=X - Run 7

Simulating for EURUSD=X - Run 8

Simulating for EURUSD=X - Run 9

Simulating for EURUSD=X - Run 10

Simulating for EURUSD=X - Run 11

Simulating for EURUSD=X - Run 12

Simulating for EURUSD=X - Run 13

Simulating for EURUSD=X - Run 14

Simulating for EURUSD=X - Run 15

Simulating for EURUSD=X - Run 16

Simulating for EURUSD=X - Run 17

Simulating for EURUSD=X - Run 18

Simulating for EURUSD=X - Run 19

Simulating for EURUSD=X - Run 20

Simulating for EURUSD=X - Run 1

Simulating for EURUSD=X - Run 2

Simulating for EURUSD=X - Run 3

Simulating for EURUSD=X - Run 4

Simulating for EURUSD=X - Run 5

Simulating for EURUSD=X - Run 6

Simulating for EURUSD=X - Run 7

Simulating for EURUSD=X - Run 8

Simulating for EURUSD=X - Run 9

Simulating for EURUSD=X - Run 1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed




Simulating for JPY=X - Run 15

Simulating for JPY=X - Run 16

Simulating for JPY=X - Run 17

Simulating for JPY=X - Run 18

Simulating for JPY=X - Run 19

Simulating for JPY=X - Run 20

Simulating for GBP=X - Run 1

Simulating for GBP=X - Run 2

Simulating for GBP=X - Run 3

Simulating for GBP=X - Run 4

Simulating for GBP=X - Run 5

Simulating for GBP=X - Run 6

Simulating for GBP=X - Run 7

Simulating for GBP=X - Run 8

Simulating for GBP=X - Run 9

Simulating for GBP=X - Run 10

Simulating for GBP=X - Run 11

Simulating for GBP=X - Run 12

Simulating for GBP=X - Run 13

Simulating for GBP=X - Run 14

Simulating for GBP=X - Run 15

Simulating for GBP=X - Run 16

Simulating for GBP=X - Run 17

Simulating for GBP=X - Run 18

Simulating for GBP=X - Run 19

Simulating for GBP=X - Run 20

Simulating for GBPAUD=X - Run 1

Simulating for GBPAUD=X - Run 2

Simulating for GBPAUD=X - Run 3

Simulating for GBPAUD=X - Run 4

Simulating for GBPAUD=X - Run 5

Simulating for GBPAUD=X - Run 6

Sim

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...


Simulating for INR=X - Run 1

Simulating for INR=X - Run 2

Simulating for INR=X - Run 3

Simulating for INR=X - Run 4

Simulating for INR=X - Run 5

Simulating for INR=X - Run 6

Simulating for INR=X - Run 7

Simulating for INR=X - Run 8

Simulating for INR=X - Run 9

Simulating for INR=X - Run 10

Simulating for INR=X - Run 11

Simulating for INR=X - Run 12

Simulating for INR=X - Run 13

Simulating for INR=X - Run 14

Simulating for INR=X - Run 15

Simulating for INR=X - Run 16

Simulating for INR=X - Run 17

Simulating for INR=X - Run 18

Simulating for INR=X - Run 19

Simulating for INR=X - Run 20

Simulating for MXN=X - Run 1

Simulating for MXN=X - Run 2

Simulating for MXN=X - Run 3

Simulating for MXN=X - Run 4

Simulating for MXN=X - Run 5

Simulating for MXN=X - Run 6

Simulating for MXN=X - Run 7

Simulating for MXN=X - Run 8

Simulating for MXN=X - Run 9

Simulating for MXN=X - Run 10

Simulating for MXN=X - Run 11

Simulating for MXN=X - Run 12

Simulating for MXN=X - Ru

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Simulating for IDR=X - Run 1

Simulating for IDR=X - Run 2

Simulating for IDR=X - Run 3

Simulating for IDR=X - Run 4

Simulating for IDR=X - Run 5

Simulating for IDR=X - Run 6

Simulating for IDR=X - Run 7

Simulating for IDR=X - Run 8

Simulating for IDR=X - Run 9

Simulating for IDR=X - Run 10

Simulating for IDR=X - Run 11

Simulating for IDR=X - Run 12

Simulating for IDR=X - Run 13

Simulating for IDR=X - Run 14

Simulating for IDR=X - Run 15

Simulating for IDR=X - Run 16

Simulating for IDR=X - Run 17

Simulating for IDR=X - Run 18

Simulating for IDR=X - Run 19

Simulating for IDR=X - Run 20

Simulating for THB=X - Run 1

Simulating for THB=X - Run 2

Simulating for THB=X - Run 3

Simulating for THB=X - Run 4

Simulating for THB=X - Run 5

Simulating for THB=X - Run 6

Simulating for THB=X - Run 7

Simulating for THB=X - Run 8

Simulating for THB=X - Run 9

Simulating for THB=X - Run 10

Simulating for THB=X - Run 11

Simulating for THB=X - Run 12

Simulating for THB=X - Ru

[*********************100%***********************]  1 of 1 completed



Simulating for RUB=X - Run 1

Simulating for RUB=X - Run 2

Simulating for RUB=X - Run 3

Simulating for RUB=X - Run 4

Simulating for RUB=X - Run 5

Simulating for RUB=X - Run 6

Simulating for RUB=X - Run 7

Simulating for RUB=X - Run 8

Simulating for RUB=X - Run 9

Simulating for RUB=X - Run 10

Simulating for RUB=X - Run 11

Simulating for RUB=X - Run 12

Simulating for RUB=X - Run 13

Simulating for RUB=X - Run 14

Simulating for RUB=X - Run 15

Simulating for RUB=X - Run 16

Simulating for RUB=X - Run 17

Simulating for RUB=X - Run 18

Simulating for RUB=X - Run 19

Simulating for RUB=X - Run 20


In [166]:
# Take averages across the simulations to get the final results
new_df = performance_df.groupby('Currency').agg({
    'Final Inventory': 'mean',
    'Final Cash': 'mean',
    'Total Trades': 'mean',
    'Profit': 'mean',
    'Profit Percentage': 'mean'
}).reset_index()

print(new_df)


    Currency  Final Inventory    Final Cash  Total Trades        Profit  \
0      CNY=X              0.0  5.250585e+08      544.3900  5.845897e+04   
1   EURCHF=X              0.0  5.250080e+08      545.1850  7.989008e+03   
2   EURHUF=X              0.0  5.282040e+08      543.7550  3.203972e+06   
3   EURJPY=X              0.0  5.262729e+08      546.3550  1.272930e+06   
4   EURSEK=X              0.0  5.250935e+08      543.7400  9.351408e+04   
5   EURUSD=X              0.0  5.250089e+08      545.6225  8.884754e+03   
6      GBP=X              0.0  5.250066e+08      546.0550  6.634933e+03   
7   GBPAUD=X              0.0  5.250154e+08      546.5700  1.543586e+04   
8   GBPCAD=X              0.0  5.250139e+08      545.2875  1.393764e+04   
9   GBPCHF=X              0.0  5.250093e+08      546.0250  9.308050e+03   
10  GBPCNY=X              0.0  5.250730e+08      545.0550  7.297047e+04   
11  GBPINR=X              0.0  5.258484e+08      545.8150  8.483995e+05   
12  GBPJPY=X             

In [167]:
# Return the currencies with any profit

profitable_currencies = new_df[new_df['Profit Percentage'] > 0]['Currency'].tolist()
print(profitable_currencies)


['CNY=X', 'EURCHF=X', 'EURHUF=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBP=X', 'GBPAUD=X', 'GBPCAD=X', 'GBPCHF=X', 'GBPCNY=X', 'GBPINR=X', 'GBPJPY=X', 'GBPNOK=X', 'GBPQAR=X', 'GBPZAR=X', 'HKD=X', 'IDR=X', 'INR=X', 'JPY=X', 'MXN=X', 'MYR=X', 'PHP=X', 'RUB=X', 'SGD=X', 'THB=X', 'ZAR=X']


In [168]:
# Return the currencies with negative or 0 profit

unprofitable_currencies = new_df[new_df['Profit'] <= 0]['Currency'].tolist()
print(unprofitable_currencies)


[]
